In [2]:
import pandas as pd
import numpy as np
import tables
import datetime as dt


exchange = 'NSE'
symbol = "HDFC"
today = dt.datetime.now().strftime("%d_%m_%y")
sample_freq = '5min'

# OAuth Login

In [3]:
from kiteconnect import KiteConnect
import platform
from selenium import webdriver
import re

#logging.basicConfig(level=logging.DEBUG)

kite = KiteConnect(api_key="b2w0sfnr1zr92nxm")
#kite.login_url()

if platform.system() == "Windows":
        driver = webdriver.Chrome("./automation/chromedriver.exe")
else:
    driver = webdriver.Chrome("./automation/chromedriver")

driver.get(kite.login_url())

In [4]:
# Redirect the user to the login url obtained
# from kite.login_url(), and receive the request_token
# from the registered redirect url after the login flow.
# Once you have the request_token, obtain the access_token
# as follows.
request_token = re.findall("request_token=(.*)&action",driver.current_url)

data = kite.generate_session(request_token[0], api_secret="jtga2mp2e5fn29h8w0pe2kb722g3dh1q")
kite.set_access_token(data["access_token"])

driver.close()

# Stock List Download

In [5]:
def getInstruments(exchange='NSE'):
    instruments_df = pd.DataFrame(data=kite.instruments(exchange))
    instruments_df = instruments_df.set_index('tradingsymbol')
    return instruments_df

In [6]:
instruments_df = getInstruments(exchange)
instruments_df.to_hdf('kite_data/kite_data.h5', key=exchange+today,mode='a')

# Historical Data Download

In [7]:
def downloadData(symbol="HDFC", fromDate= dt.datetime.now() - dt.timedelta(days = 1), toDate=dt.datetime.now(), freq="minute"):
    symbolToken = instruments_df.loc[symbol,'instrument_token']
    raw_data = pd.DataFrame(data=kite.historical_data(symbolToken, fromDate, toDate, freq, continuous=False))
    raw_data = raw_data.set_index('date').tz_localize(None)
    return raw_data
                

In [18]:
def resample(data,freq):
    data = data.resample(freq).agg({'open':'first','high':'max','low':'min','close':'last','volume':'sum'})
    #data.columns = data.columns.droplevel()
    return data
    
    

In [9]:
raw_data = downloadData()

raw_data.to_hdf('kite_data/kite_data.h5', key=symbol+today,mode='a')

In [19]:
resample(raw_data, sample_freq)

,open,high,low,close,volume
date,,,,,
2019-04-05 09:15:00,2045.00,2048.05,2035.00,2047.55,109213
2019-04-05 09:20:00,2047.45,2056.00,2047.05,2055.70,111520
2019-04-05 09:25:00,2055.30,2059.00,2054.15,2057.00,58952
2019-04-05 09:30:00,2057.40,2057.40,2051.15,2051.55,47270
2019-04-05 09:35:00,2050.05,2051.80,2049.00,2050.70,78126
2019-04-05 09:40:00,2051.00,2053.15,2050.00,2050.70,31274
2019-04-05 09:45:00,2050.90,2050.90,2045.55,2046.10,25359
2019-04-05 09:50:00,2046.70,2050.35,2046.10,2049.00,122384
2019-04-05 09:55:00,2049.00,2049.95,2048.00,2049.00,14511


# Filter Stocks

In [12]:
raw_data

,close,high,low,open,volume
date,,,,,
2019-04-05 09:15:00,2040.05,2048.05,2040.00,2045.00,37267
2019-04-05 09:16:00,2037.05,2040.25,2035.10,2040.00,25499
2019-04-05 09:17:00,2035.85,2037.55,2035.00,2036.55,18412
2019-04-05 09:18:00,2038.80,2038.80,2035.00,2035.90,7742
2019-04-05 09:19:00,2047.55,2047.55,2038.80,2038.80,20293
2019-04-05 09:20:00,2048.35,2051.00,2047.05,2047.45,34723
2019-04-05 09:21:00,2048.95,2049.90,2047.75,2048.30,18310
2019-04-05 09:22:00,2052.50,2053.30,2049.10,2049.10,29982
2019-04-05 09:23:00,2053.00,2055.35,2051.75,2052.50,15575


# Live Data resampling

# Indicators calculation

# Algorithm

# Visualization

# WebSocket Handler

In [ ]:
import multiprocessing

def worker():
    """worker function"""
    print 'Worker'
    return

if __name__ == '__main__':
    jobs = []
    for i in range(5):
        p = multiprocessing.Process(target=worker)
        jobs.append(p)
        p.start()